Extracting a DataFrame - Weather-Based Clothing Recommender

## Introduction

This deliverable aims to create a **DataFrame** for a system that provides **clothing recommendations based on weather data**. The system utilizes the **OpenWeatherMap API** to retrieve weather information for specific cities and generates personalized clothing suggestions based on factors such as **temperature, perceived temperature, rainfall, wind speed, and cloud coverage**. Additionally, the functionality is extended to include **hourly recommendations for today and the next 5 days**.

The project has practical applications in industries such as **tourism, fashion, and daily planning services**, providing a smart approach to daily preparation and personal logistics.

---

## Objectives

1. **Integration with OpenWeatherMap**:
   - Retrieve weather data for a list of cities, including temperature, perceived temperature, rainfall, wind, and cloudiness.
   - Obtain forecasts for today and the next 5 days, broken down into hourly segments.

2. **Generate clothing recommendations**:
   - Provide clothing suggestions based on perceived temperature and rainfall probability.

3. **Data storage and export**:
   - Create a **DataFrame** with the collected data and export it in **CSV** and **Excel** formats, including timestamps for easier analysis.

---

## System Development

### **Requirements**

- **Libraries**: `requests`, `pandas`, `datetime`
- **Preliminary Setup**:
  - Register an account on OpenWeatherMap and obtain an API key.
  - Store the API key as an environment variable for security.

---

### **Development Steps**

#### **1. Defining the Recommendation Function**
The `get_clothing_recommendation` function evaluates two main factors:
- **Perceived temperature (`feels_like`)**: Adjusts the recommendation based on how cold or warm it feels.
- **Rain (`rain`)**: Includes waterproof clothing in case of precipitation.

#### **2. Retrieving Hourly Forecast**
The `get_weather_data_next_days` function was designed to interact with the OpenWeatherMap API and obtain:
- Weather data for today and the next 5 days.
- Specific hourly segments (morning, afternoon, evening, night).
- Additional variables such as temperature, wind, cloudiness, and rainfall.

For each time segment, specific recommendations are assigned using the recommendation function.

#### **3. Handling Time Zones**
Deprecated methods (`utcfromtimestamp` and `utcnow`) were replaced with **timezone-aware objects** to ensure compatibility with future Python versions and correctly handle time zone differences.

#### **4. Data Export**
The collected and processed data is stored in a **Pandas DataFrame**, which is then exported in **CSV** and **Excel** formats, with filenames including the current date and time for easy identification.

---

## **Results**

1. **Generated Data**:
   - Clothing recommendations based on hourly segments for over **200 cities** (main cities in Spain, Europe, and other major tourist regions).
   - Consideration of multiple weather variables: temperature, perceived temperature, rainfall, wind speed, and cloud coverage.

2. **Exported Files**:
   - `weather_clothing_recommendations_<date>_<time>.csv`
   - `weather_clothing_recommendations_<date>_<time>.xlsx`

3. **Example Output**:
   A fragment of the DataFrame includes columns such as:
   - City
   - Hourly segment
   - Temperature
   - Perceived temperature
   - Wind speed
   - Cloud coverage
   - Clothing recommendation

---

# **Business Application of the Project**
This project provides a solid foundation for developing a **personalized weather-based service**, with various commercial applications, including:

1. **Personalized Clothing Recommender**:
   - Provide specific clothing suggestions based on local weather conditions and hourly forecasts.
   - Potential integration with marketplaces like **Amazon or Zalando** to directly recommend products.

2. **Service for the Tourism Sector**:
   - Generate **customized clothing and activity recommendations** for tourists based on weather conditions at their destination.

3. **Integrated E-Commerce Platform**:
   - Enhance customer experience by suggesting weather-related products **directly in an online store**.

4. **Weather Assistant for Professionals & Events**:
   - Provide **detailed forecasts and clothing recommendations** for event organizers or outdoor workers.

---

## **Conclusion**
The developed system offers a comprehensive approach to **daily outfit planning**, considering detailed weather variables. This not only simplifies users' daily lives but also has the potential for integration into commercial applications in **tourism, retail fashion, and smart planning services**.


In [56]:
import os
from datetime import datetime, timedelta, timezone
import requests
import pandas as pd

In [77]:
# Set up the API key as an environment variable
API_KEY = os.getenv("OpenWeather")
if not API_KEY:
    raise ValueError("The OpenWeatherMap API key is not set as an environment variable.")


In [60]:
# Dictionary for clothing recommendations based on perceived temperature and rain
def get_clothing_recommendation(feels_like, rain):
    if rain:
        if feels_like < 0:
            return "Heavy coat, scarf, gloves, thermal boots, and raincoat"
        elif 0 <= feels_like < 10:
            return "Coat, scarf, winter clothing, and raincoat"
        elif 10 <= feels_like < 20:
            return "Light jacket, mid-season clothing, and raincoat"
        elif 20 <= feels_like < 30:
            return "Light clothing, raincoat, or umbrella"
        else:
            return "Very light clothing, cap, sunglasses, and umbrella"
    else:
        if feels_like < 0:
            return "Heavy coat, scarf, gloves, and thermal boots"
        elif 0 <= feels_like < 10:
            return "Coat, scarf, and winter clothing"
        elif 10 <= feels_like < 20:
            return "Light jacket and mid-season clothing"
        elif 20 <= feels_like < 30:
            return "Light clothing such as t-shirts and shorts"
        else:
            return "Very light clothing, cap, and sunglasses"


In [79]:
def get_weather_data_next_days(city):
    try:
        forecast_url = f"https://api.openweathermap.org/data/2.5/forecast?q={city}&appid={API_KEY}&units=metric"
        forecast_response = requests.get(forecast_url)
        forecast_response.raise_for_status()
        forecast_data = forecast_response.json()

        # Create a data structure for each day and time slot
        weather_data = {}
        current_date = datetime.now(timezone.utc).date()
        cutoff_date = current_date + timedelta(days=3)  # Limit to the next 3 days

        for forecast in forecast_data["list"]:
            date = datetime.fromtimestamp(forecast["dt"], tz=timezone.utc).date()
            if date > cutoff_date:  # Filter out dates beyond 3 days
                continue

            hour = datetime.fromtimestamp(forecast["dt"], tz=timezone.utc).hour
            day_label = f"Day {((date - current_date).days + 1)}" if date > current_date else "Today"

            period = (
                "Morning" if 6 <= hour < 12 else
                "Afternoon" if 12 <= hour < 18 else
                "Evening" if 18 <= hour < 24 else "Night"
            )

            key = (day_label, period, hour)

            weather_data[key] = {
                "temp": forecast["main"]["temp"],
                "feels_like": forecast["main"]["feels_like"],
                "wind_speed": forecast["wind"]["speed"],
                "cloudiness": forecast["clouds"]["all"],
                "rain": "rain" in forecast["weather"][0]["main"].lower(),
                "condition": forecast["weather"][0]["description"]
            }

        # Create final data with specific recommendations
        final_data = {"City": city}
        for (day, period, hour), data in weather_data.items():
            temp = data["temp"]
            feels_like = data["feels_like"]
            wind_speed = data["wind_speed"]
            cloudiness = data["cloudiness"]
            rain = data["rain"]
            condition = data["condition"]

            # Generate dynamic keys for time slot data
            final_data[f"{day} {period} ({hour}:00) Temperature (°C)"] = temp
            final_data[f"{day} {period} ({hour}:00) Feels Like (°C)"] = feels_like
            final_data[f"{day} {period} ({hour}:00) Wind Speed (m/s)"] = wind_speed
            final_data[f"{day} {period} ({hour}:00) Cloudiness (%)"] = cloudiness
            final_data[f"{day} {period} ({hour}:00) Rain"] = rain
            final_data[f"{day} {period} ({hour}:00) Condition"] = condition
            final_data[f"{day} {period} ({hour}:00) Recommendation"] = get_clothing_recommendation(feels_like, rain)

        return final_data

    except requests.exceptions.RequestException as e:
        print(f"Error retrieving weather data for {city}: {e}")
        return None


In [81]:
# List of the most popular tourist cities in Europe and Spain
cities = [
    # North America
    "New York", "Los Angeles", "Miami", "Orlando", "Las Vegas", "Chicago", "San Francisco", "Toronto", "Vancouver", "Montreal",
    "Mexico City", "Cancun", "Playa del Carmen", "Tulum", "Puerto Vallarta", "Los Cabos", "Guadalajara", "Monterrey", "Merida", "Puebla",
    # South America
    "Rio de Janeiro", "São Paulo", "Buenos Aires", "Lima", "Cusco", "Cartagena", "Bogotá", "Medellín", "Santiago", "Valparaíso",
    # Europe
    "Paris", "London", "Rome", "Venice", "Amsterdam", "Berlin", "Florence", "Prague", "Vienna", "Budapest",
    "Madrid", "Barcelona", "Seville", "Valencia", "Granada", "Bilbao", "Málaga", "Córdoba", "San Sebastián", "Zaragoza",
    # Other destinations in Spain
    "Toledo", "Segovia", "Santiago de Compostela", "Palma de Mallorca", "Alicante", "Las Palmas de Gran Canaria",
    # More global destinations...
]

# Retrieve weather data and clothing recommendations for each city
data = []
for city in cities:
    weather_data = get_weather_data_next_days(city)  # Use the updated function
    if weather_data:
        data.append(weather_data)

# Create a DataFrame with the retrieved data
df = pd.DataFrame(data)

# Generate file names with date and time
current_time = datetime.now().strftime("%Y-%m-%d_%H-%M")
csv_filename = f"weather_forecast_recommendations_{current_time}.csv"
excel_filename = f"weather_forecast_recommendations_{current_time}.xlsx"

# Save the DataFrame as CSV and Excel files
df.to_csv(csv_filename, index=False)
df.to_excel(excel_filename, index=False)

# Display results
print("Clothing recommendations saved in the following files:")
print(f"- {csv_filename}\n- {excel_filename}")
print(df)


Recomendaciones de ropa guardadas en los siguientes archivos:
- weather_forecast_recommendations_2024-12-15_22-22.csv
- weather_forecast_recommendations_2024-12-15_22-22.xlsx
                    City  Day 2 Night (0:00) Temperature (°C)  \
0               New York                                 4.14   
1            Los Angeles                                17.45   
2                  Miami                                24.45   
3                Orlando                                22.98   
4              Las Vegas                                15.58   
..                   ...                                  ...   
421  Alcázar de San Juan                                 4.82   
422           Valdepeñas                                 4.74   
423         Vélez-Málaga                                13.59   
424            Plasencia                                 5.11   
425             Trujillo                                18.99   

     Day 2 Night (0:00) Feels Like (°C)  Day